In [1]:
from flask import Flask, render_template, request, redirect, url_for, flash, jsonify
from pymongo import MongoClient
import pandas as pd
import joblib
import requests
import os
from bson import ObjectId
from io import BytesIO

app = Flask(__name__)
app.secret_key = "supersecretkey"

# MongoDB Setup
client = MongoClient("mongodb://localhost:27017/")
db = client["solar_energy_db"]
weather_collection = db["weather"]
metadata_collection = db["metadata"]
solarproduction_collection = db["solarproduction"]

# Load the ML Model
model = joblib.load("best_model.pkl")

@app.route('/')
def index():
    return render_template("index.html")

# Metadata Management
@app.route('/metadata', methods=['GET', 'POST'])
def manage_metadata():
    if request.method == 'POST':
        data = request.form.to_dict()
        metadata_collection.insert_one(data)
        flash("Metadata added successfully!", "success")
        return redirect(url_for('manage_metadata'))
    
    metadata = list(metadata_collection.find())
    return render_template("metadata.html", metadata=metadata)

@app.route('/delete_metadata/<id>')
def delete_metadata(id):
    metadata_collection.delete_one({"_id": ObjectId(id)})
    flash("Metadata deleted successfully!", "danger")
    return redirect(url_for('manage_metadata'))

# Forecasting and Weather Data Handling
@app.route('/forecast', methods=['GET', 'POST'])
def forecast():
    if request.method == 'POST':
        if request.form['submit'] == "Fetch & Predict":
            start_date = request.form['start_date']
            end_date = request.form['end_date']
            pv_serial = request.form['pv_serial']

            # Fetch data from Open-Meteo API
            api_url = f"https://api.open-meteo.com/v1/forecast?...&start_date={start_date}&end_date={end_date}"
            response = requests.get(api_url)
            weather_data = response.json()

            # Store in MongoDB
            for entry in weather_data['data']:
                entry['PV Serial Number'] = pv_serial
                entry['weather_flag'] = 'Forecast'
                weather_collection.insert_one(entry)

            # Predict solar output
            df = pd.DataFrame(weather_data['data'])
            features = df[['temperature', 'shortwave_radiation', 'relative_humidity']]
            predictions = model.predict(features)

            for i, pred in enumerate(predictions):
                solarproduction_collection.insert_one({
                    "PV Serial Number": pv_serial,
                    "Date": df.loc[i, 'date'],
                    "Predicted Energy (kWh)": pred,
                    "solarFlag": "Prediction"
                })
        else:
            plant_name = request.form['plant_name']
            file = request.files['file']
            df = pd.read_csv(file,skiprows=3)
            df, data_dict = data_processing(df)
            mongo_upload(data_dict)
            result = data_transform(df)
            metadata = mongo_fetch(plant_name)
            res_df = model_prediciton(result,metadata)
            
            
            output = BytesIO()
            writer = pd.ExcelWriter(output, engine='xlsxwriter')
            df.to_excel(writer, sheet_name='Sheet1')
            writer.save()
            xlsx_data = output.getvalue()

            return send_file(xlsx_data, attachment_filename='output.xlsx', as_attachment=True)    

        
        flash("Forecast data and predictions saved successfully!", "success")
        return redirect(url_for('forecast'))
    
    solardata = list(solarproduction_collection.find())
    return render_template("forecast.html",solardata=solardata)

# Actual Solar Output Upload
@app.route('/actual_output', methods=['GET', 'POST'])
def actual_output():
    if request.method == 'POST':
        pv_serial = request.form['pv_serial']
        upload_type = request.form['upload_type']

        if upload_type == 'bulk':
            file = request.files['file']
            df = pd.read_excel(file)
            for _, row in df.iterrows():
                solarproduction_collection.insert_one({
                    "PV Serial Number": pv_serial,
                    "Date": row['date'],
                    "Actual Energy (kWh)": row['actual_output'],
                    "solarFlag": "Actual"
                })
        else:
            date = request.form['date']
            actual_output = request.form['actual_output']
            solarproduction_collection.insert_one({
                "PV Serial Number": pv_serial,
                "Date": date,
                "Actual Energy (kWh)": actual_output,
                "solarFlag": "Actual"
            })

        flash("Actual solar output saved successfully!", "success")
        return redirect(url_for('actual_output'))

    return render_template("actual_output.html")

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Feb/2025 23:18:14] "GET /forecast HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2025 23:18:14] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [28/Feb/2025 23:18:35] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [28/Feb/2025 23:19:54] "GET /forecast HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2025 23:19:54] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [28/Feb/2025 23:19:59] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [28/Feb/2025 23:20:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2025 23:20:37] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [28/Feb/2025 23:20:37] "GET /metadata HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2025 23:20:37] "GET /static/styles.css HTTP/1.1" 304 -
